In [14]:
import pandas as pd

# Ler o CSV
df = pd.read_csv('comentarios.csv')

# Visualizar os dados
df = df[['product_name', 'review_text', 'product_brand', 'site_category_lv1', 'site_category_lv2']]  # Selecionar colunas

df['review_text'] = df['review_text'].astype(str)  # Garantir que os comentários sejam strings

# Salvar os dados limpos
df.to_csv('dados_processados.csv', index=False)


C:\Users\Douglas\AppData\Local\Temp\ipykernel_14856\2087078591.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('comentarios.csv')


In [16]:
from sentence_transformers import SentenceTransformer
import chromadb
import pandas as pd

# Carregar o modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Configurar o cliente ChromaDB
client = chromadb.Client()

# Criar uma coleção para armazenar os embeddings (ou conectar à coleção já existente)
collection = client.get_or_create_collection(name="comentarios")

# Definir o tamanho do chunk (ex: 1000 linhas por vez)
chunk_size = 500

# Ler o CSV em chunks e processar
for chunk in pd.read_csv('dados_processados.csv', chunksize=chunk_size):
    chunk = chunk.dropna(subset=['review_text'])  # Remover valores nulos
    chunk['review_text'] = chunk['review_text'].astype(str)  # Garantir que os comentários sejam strings

    # Transformar comentários em uma lista
    comentarios = chunk['review_text'].tolist()

    # Gerar embeddings em chunks
    embeddings = model.encode(comentarios)

    # Adicionar embeddings e metadados no ChromaDB
    collection.add(
        embeddings=embeddings.tolist(), 
        metadatas=chunk.to_dict('records'),  # Inserir os dados do chunk como metadados
        ids=[str(i) for i in range(len(comentarios))]  # Usar índices como IDs
    )

print("Todos os chunks foram processados e armazenados.")


c:\Users\Douglas\Desktop\ia\ambiente_virtual\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 4
Add of existing embedding ID: 5
Add of existing embedding ID: 6
Add of existing embedding ID: 7
Add of existing embedding ID: 8
Add of existing embedding ID: 9
Add of existing embedding ID: 10
Add of existing embedding ID: 11
Add of existing embedding ID: 12
Add of existing embedding ID: 13
Add of existing embedding ID: 14
Add of existing embedding ID: 15
Add of existing embedding ID: 16
Add of existing embedding ID: 17
Add

Todos os chunks foram processados e armazenados.


In [18]:
from sentence_transformers import SentenceTransformer
import chromadb
import pandas as pd

# Carregar o modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')  

# Lê o arquivo CSV contendo os dados processados
df = pd.read_csv('dados_processados.csv')  

# Configurar o cliente ChromaDB
client = chromadb.Client()

# Conectar à coleção de comentários já inseridos
collection = client.get_collection(name="comentarios")

# Função para detectar uma categoria ou produto na query do usuário
def detectar_categoria_dinamicamente(query, df):
    query = query.lower()  # Converte a query para minúsculas para comparação
    for coluna in ['product_name', 'product_brand', 'site_category_lv1', 'site_category_lv2']:
        valores_unicos = df[coluna].dropna().unique()  # Remove valores nulos e extrai valores únicos da coluna
        for valor in valores_unicos:
            if valor.lower() in query:  # Verifica se o valor aparece na query
                return valor  # Se encontrado, retorna o valor
    return None  # Se nada for encontrado, retorna None

# Função para buscar comentários com base na query do usuário
def buscar_comentarios_dinamicamente(query, df):
    categoria = detectar_categoria_dinamicamente(query, df)  # Chama a função para detectar a categoria ou produto
    if categoria:
        print(f"Categoria ou produto detectado: {categoria}")
        df_filtrado = df[(df['product_name'].str.contains(categoria, case=False, na=False)) |
                         (df['product_brand'].str.contains(categoria, case=False, na=False)) |
                         (df['site_category_lv1'].str.contains(categoria, case=False, na=False)) |
                         (df['site_category_lv2'].str.contains(categoria, case=False, na=False))].copy()
    else:
        print("Nenhuma categoria ou produto detectado, usando todos os produtos.")  
        df_filtrado = df.copy()  
    return df_filtrado

# Função para buscar a resposta automaticamente com base na query e nos dados
def buscar_resposta_automatica(query, df, model, collection):
    df_filtrado = buscar_comentarios_dinamicamente(query, df)  # Filtra os comentários com base na query
    
    if df_filtrado.empty:
        return "Nenhum comentário encontrado para essa categoria ou produto."
    
    # Garantir que todos os comentários sejam strings
    df_filtrado['review_text'] = df_filtrado['review_text'].astype(str)
    
    comentarios_filtrados = df_filtrado['review_text'].tolist()
    
    if not comentarios_filtrados:
        return "Nenhum comentário disponível para busca."  
    
    query_embedding = model.encode([query])

    # Busca no ChromaDB os 2 comentários mais próximos da query
    resultados = collection.query(
        query_embeddings=query_embedding.tolist(),
        n_results=2
    )

    # Aqui ajustamos a iteração para acessar os metadados corretamente
    comentarios_encontrados = [result['review_text'] for result in resultados['metadatas'][0]]

    return comentarios_encontrados

query = "melhor celular ?"
resultados = buscar_resposta_automatica(query, df, model, collection)

# Exibe os resultados encontrados
for resultado in resultados:
    print(resultado)


c:\Users\Douglas\Desktop\ia\ambiente_virtual\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Categoria ou produto detectado: Celular
Melhor custo benefício. Exatamente  como anunciado. Recomendo .
Gostei do celular. Boa tela sem ser um celular gigante. O processador poderia ser melhor para a segunda linha da LG, porém o preço seria maior também. Eu o recomendo.
